# __Predicting cell biological response__

In [45]:
import wandb
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import logging

#### Run other .ipybn files

In [57]:
# %run NN_model.ipynb
from NN_model import *

%run NN_dataset.ipynb

         D1        D2    D3   D4        D5        D6        D7        D8  \
0  0.000000  0.497009  0.10  0.0  0.132956  0.678031  0.273166  0.585445   
1  0.366667  0.606291  0.05  0.0  0.111209  0.803455  0.106105  0.411754   
2  0.033300  0.480124  0.00  0.0  0.209791  0.610350  0.356453  0.517720   
3  0.000000  0.538825  0.00  0.5  0.196344  0.724230  0.235606  0.288764   
4  0.100000  0.517794  0.00  0.0  0.494734  0.781422  0.154361  0.303809   

         D9       D10  ...  D1768  D1769  D1770  D1771  D1772  D1773  D1774  \
0  0.743663  0.243144  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1  0.836582  0.106480  ...    1.0    1.0    1.0    0.0    1.0    0.0    0.0   
2  0.679051  0.352308  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3  0.805110  0.208989  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4  0.812646  0.125177  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

   D1775  D1776  target  
0    0.0    0.0       1  
1    1.0    0.0 

#### Dataset representation for NN model

In [47]:
class CellDataset(Dataset):
    def __init__(self, data: pd.DataFrame, normalize: bool):
        # Save predictors as DataFrame
        self.cell_descriptors = data.drop(columns=['target'])

        # Normalize target value to [-1, 1]
        res = data['target']
        if normalize:
            res = res.apply(lambda x: -1 if x == 0 else 1)

        # Save target as DataFrame
        self.cell_response = res.astype('float64').to_frame()

    def __len__(self):
        return len(self.cell_descriptors)

    def __getitem__(self, idx):
        desc = self.cell_descriptors.iloc[idx]
        res = self.cell_response.iloc[idx]
        return desc.values, res.values
    
    def get_input_size(self):
        return self.cell_descriptors.shape[1]

## Wandb Project

In [26]:
wandb.login()

%env "WANDB_NOTEBOOK_NAME" "NN-z1"

run = wandb.init(
    entity = "matus13579",  #dont change
    project = "NN-z1",      #dont change
    name = "test_run"       #run name
    #id =                   #define run with ID (used for resuming)
    #resume = True           #resume run
    )

#show graphs in Jupyter Notebook
#%%wandb                   

logger = logging.getLogger("wandb")
logger.setLevel(logging.ERROR)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


env: "WANDB_NOTEBOOK_NAME"="NN-z1"


## Experiments

In [48]:
data = pd.read_csv('bioresponse.csv')

#### Load Dataset and Initialize NN model

In [50]:
def setup_experiment(current_data):
    # Split dataset -> 20% testing, 80% training
    # Stratified split = each dataset has equal amounts of each class (saved in column 'target')
    train, test = train_test_split(current_data, test_size = 0.2, stratify = current_data['target']) 

    # Initlize dataset for NN
    train_data = CellDataset(train, normalize_dataset())
    test_data = CellDataset(test, normalize_dataset())

    # Create NN and training class
    mlp = MultiLayerPerceptron(train_data.get_input_size())
    trainer = Trainer(config, mlp)

    # Load dataset
    trainer.load_dataset(train_data, test_data)

    return trainer

### Training loop

In [52]:
def run_experiment(myrun, trainer: Trainer, no_epochs):
    best_model = None
    best_accuracy = None

    for epoch in range(no_epochs):
        print (f"Epoch {epoch}")

        # Train model
        trainer.train()

        # Get metrics
        metrics = trainer.evaluate()

        loss_tr, loss_val = trainer.mean_loss()

        print (f"loss_training : {loss_tr} | loss_training : {loss_val}")

        myrun.log({"loss_training": loss_tr})
        myrun.log({"loss_validate": loss_val})
        myrun.log({"accuracy": metrics.accuracy})
        myrun.log({"f1_score": metrics.accuracy})

        if (best_accuracy is None) or (best_accuracy > metrics.accuracy):
            best_accuracy = metrics.accuracy
            best_model = trainer.model


#### Experiment 1

In [53]:
current_data = correlation_selection_merged(data)

config.activation_fn = "sigmoid"

C:\Users\matul\AppData\Local\Temp\ipykernel_24224\1673334171.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data[merged_descriptor_name] = (data[descriptor_pair[0]] + data[descriptor_pair[1]]) / 2
C:\Users\matul\AppData\Local\Temp\ipykernel_24224\1673334171.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data[merged_descriptor_name] = (data[descriptor_pair[0]] + data[descriptor_pair[1]]) / 2
C:\Users\matul\AppData\Local\Temp\ipykernel_24224\1673334171.py:32: PerformanceWarning: DataFrame is highly fragme

Selected Features from merged data:
['D27', 'D469', 'D146_D596', 'D182_D595']


In [56]:
current_trainer = setup_experiment(current_data)
print (current_trainer.loss_train())
print (current_trainer.loss_validate())

Using cpu device for training


AttributeError: 'Trainer' object has no attribute 'loss_train'

In [54]:
current_trainer = setup_experiment(current_data)
run_experiment(run, current_trainer, 10)

Using cpu device for training
Epoch 0


AttributeError: 'Trainer' object has no attribute 'mean_loss'